In [18]:
import requests
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import os
from openai import OpenAI
import tiktoken

In [19]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## Q1. Running Elastic

In [20]:
es_client = Elasticsearch('http://localhost:9200/')
es_client.info()

ObjectApiResponse({'name': '08674c4bedf4', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'WwSOB8B5S06XQCsZh0AKzw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

## Q2. Indexing the data

In [23]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

/var/folders/nq/356h8mk15m537mvwf_mf87t80000gn/T/ipykernel_54757/295183685.py:18: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [24]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

## Q3. Searching

In [25]:
query = 'How do execute a command on a Kubernetes pod?'

In [26]:
search_query = {
  "size" : 5,
  "query": {
    "bool": {
      "must": [
        {
          "multi_match": {
            "query": query,
            "fields": [ "question^4", "text"],
            "type": "best_fields"
          }
        }
      ]
    }
  }
}

In [27]:
response = es_client.search(index=index_name, body=search_query)
response['hits']['hits'][0]['_score']

/var/folders/nq/356h8mk15m537mvwf_mf87t80000gn/T/ipykernel_54757/3682695899.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


44.50556

## Q4. Filtering

In [28]:
query = 'How do copy a file to a Docker container?'

In [29]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [30]:
response = es_client.search(index=index_name,body=search_query)
response['hits']['hits'][2]['_source']['question']

/var/folders/nq/356h8mk15m537mvwf_mf87t80000gn/T/ipykernel_54757/2048131909.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name,body=search_query)


'How do I copy files from a different folder into docker container’s working directory?'

## Q5. Building a prompt

In [31]:
def build_prompt(query, search_results):
    context_template = """
Q: {question}
A: {text}
""".strip()
    
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    
    for doc in search_results['hits']['hits']:
        question=doc['_source']['question']
        text=doc['_source']['text']
        context+=context_template.format(question=question,text=text)+'\n\n'
    context=context.strip()
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt.strip()

In [32]:
query = 'How do copy a file to a Docker container?'
prompt = build_prompt(query,response)
len(prompt)

1446

In [33]:
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [34]:
print(llm(prompt))

You can copy a file to a Docker container using the `docker cp` command. The basic syntax for copying a file or directory from your local machine into a running Docker container is:

```
docker cp /path/to/local/file_or_directory container_id:/path/in/container
```


## Q6. Tokens

In [35]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [36]:
len(encoding.encode(prompt))

320